In [20]:
key="sk-None-DrTg56zyFxNqYnjIuqmLT3BlbkFJSpDsivlHJvDpHISqn5mz"

In [21]:
!pip install openai --upgrade

In [22]:
import os
import openai
from openai import OpenAI

In [23]:
openai.api_key=key
client=OpenAI(api_key=key)

In [24]:
!pip install pdfplumber

In [25]:
import csv
import pdfplumber
import requests
from bs4 import BeautifulSoup
import openai  # Make sure to have openai installed and configured
import re


def extract_text_from_pdfs(pdf_paths):
    all_text = ""
    for pdf_path in pdf_paths:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
    return all_text

def extract_text_from_urls(urls):
    all_text = ""
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text += soup.get_text() + "\n"
    return all_text

def generate_one_question(chunk, max_tokens=150):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert on cybersecurity."},
            {"role": "user", "content": f"Generate a multiple-choice question with four options and indicate the correct answer based on the following text:\n\n{chunk}"}
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def save_question_to_csv(questions, filename='questions.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()

        for question in questions:
            try:
                # Extract the question text
                question_match = re.match(r'Question \d+: (.+)', question)
                question_text = question_match.group(1).strip() if question_match else None

                # Extract the options
                options_match = re.findall(r'([A-D])\)\s*(.+)', question)
                options = [opt[1].strip() for opt in options_match]

                # Extract the correct answer
                correct_answer_match = re.search(r'Correct Answer:\s*([A-D])\)\s*(.+)', question)
                correct_option_letter = correct_answer_match.group(1).strip() if correct_answer_match else None
                correct_option_text = correct_answer_match.group(2).strip() if correct_answer_match else None

                if len(options) != 4:
                    print(f"Unexpected number of options: {options}. Skipping...")
                    continue

                # Ensure correct option letter matches the correct option text
                if correct_option_letter and correct_option_text:
                    correct_option_full = f"{correct_option_letter}) {correct_option_text}"
                    if correct_option_full not in options:
                        print(f"Correct answer {correct_option_full} not found in options: {options}. Skipping...")
                        continue

                # Map options to A, B, C, D
                option_a, option_b, option_c, option_d = options

                # Write the row to the CSV file
                writer.writerow({
                    'Question': question_text,
                    'Option A': option_a,
                    'Option B': option_b,
                    'Option C': option_c,
                    'Option D': option_d,
                    'Correct Answer': correct_option_letter
                })
            except (IndexError, ValueError) as e:
                print(f"Error processing question: {question}. Error: {e}")






In [26]:
pdf_paths = [
    '/content/ATTACK_Design_and_Philosophy_March_2020.pdf',
    '/content/CELEX_32016R0679_EN_TXT.pdf',
    '/content/diamond.pdf',
    '/content/nist.sp.800-150.pdf'
]
urls = [
    'https://oasis-open.github.io/cti-documentation/',
    'https://capec.mitre.org'
]

pdf_text = extract_text_from_pdfs(pdf_paths)
url_text = extract_text_from_urls(urls)
combined_text = pdf_text + url_text
text_chunks = [combined_text[i:i + 2000] for i in range(0, len(combined_text), 2000)]  # Adjust chunk size as needed


In [ ]:
import time

# Determine chunk size and number of questions
chunk_size = 2000  # Adjust chunk size as needed
text_length = len(combined_text)
num_questions = 2500  # Define how many questions you want to generate

# Create chunks of text
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, text_length, chunk_size)]

# Generate questions
questions = []
for i in range(num_questions):
    start_idx = (i * chunk_size) % text_length

    # Wrap around text if needed
    if start_idx + chunk_size <= text_length:
        chunk = combined_text[start_idx: start_idx + chunk_size]
    else:
        chunk = combined_text[start_idx:] + combined_text[:(start_idx + chunk_size) % text_length]

    print(f"Generating question {i + 1}/{num_questions}")
    response = generate_one_question(chunk) # Store the entire response
    questions.append(response)

    # Adding delay to respect rate limits
    time.sleep(1)

# Print the generated questions to inspect their structure
for i, question in enumerate(questions):
    print(f"Question {i + 1}: {question}")
    print("-" * 20) # Add a separator for clarity

Streaming output truncated to the last 5000 lines.
Correct Answer: A) The data subject is already aware of the information.
--------------------
Question 1944: What right does the data subject have to rectify inaccurate personal data according to the text provided?
A) Right to be informed of data recipients
B) Right to request data storage period
C) Right to lodge a complaint with a supervisory authority
D) Right to obtain rectification of personal data

Correct answer: D) Right to obtain rectification of personal data
--------------------
Question 1945: Multiple-choice question:
Which of the following is NOT a valid ground for requesting the erasure of personal data under Article 17 of the GDPR?
a) The personal data are no longer necessary for the purposes they were collected
b) The data subject withdraws consent for processing
c) The personal data have been collected in relation to the offer of information society services
d) The personal data have been lawfully processed

Correct an

In [28]:
print(questions)

['Which of the following statements about MITRE ATT&CK is true based on the provided text?\n\nA) MITRE ATT&CK is a closed knowledge base only accessible to government agencies.\nB) MITRE ATT&CK is primarily focused on developing cybersecurity products.\nC) MITRE ATT&CK provides a common taxonomy for offense but not defense.\nD) MITRE ATT&CK is used as a foundation for threat models in various sectors, including cybersecurity. \n\nCorrect Answer: D) MITRE ATT&CK is used as a foundation for threat models in various sectors, including cybersecurity.', 'Which organization is responsible for maintaining and evolving the ATT&CK framework?\n\nA) NSA (National Security Agency)\nB) FBI (Federal Bureau of Investigation)\nC) DHS (Department of Homeland Security)\nD) MITRE Corporation\n\nCorrect answer: D) MITRE Corporation', 'Which section of the document provides details on the relationship between ATT&CK objects?\n\nA) 3.4.1 Procedures\nB) 3.7 Mitigations\nC) 3.8 ATT&CK Object Model Relationshi

In [29]:
def parse_question(question_text):
    # Ensure the question is formatted consistently
    if 'Correct Answer' in question_text:
        parts = question_text.split('\n')
        question = parts[0].strip()
        correct_answer = ""
        options = [part.strip() for part in parts[1:] if part.strip()]

        if len(options) >= 4:
            # Extract options A, B, C, D
            option_a = options[0] if len(options) > 0 else ""
            option_b = options[1] if len(options) > 1 else ""
            option_c = options[2] if len(options) > 2 else ""
            option_d = options[3] if len(options) > 3 else ""

            # Find the correct answer
            for option in options:
                if "Correct Answer" in option:
                    correct_answer = option.split(':')[-1].strip()
                    break

            return [question, option_a, option_b, option_c, option_d, correct_answer]
        else:
            return [question_text, "", "", "", "", ""]
    else:
        return [question_text, "", "", "", "", ""]

def save_questions_to_csv(questions, filename='questions.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer'])
        for question in questions:
            parsed_question = parse_question(question)
            writer.writerow(parsed_question)


In [ ]:
questions = []
chunk_size = 2000
num_questions = 2500

text_length = len(combined_text)
for i in range(num_questions):
    start_idx = (i * chunk_size) % text_length
    chunk = combined_text[start_idx: start_idx + chunk_size]
    print(f"Generating question {i + 1}/{num_questions}")
    question = generate_one_question(chunk)
    questions.append(question)
    time.sleep(1)  # Adding delay to respect rate limits

save_questions_to_csv(questions)
print("Questions generated and saved to CSV.")